In [2]:
import tensorflow as tf
import matplotlib.pyplot as plt
import numpy as np
from numpy.linalg import eig
import pandas as pd
from minisom import MiniSom  
import math
import ipynb
from sklearn.metrics import classification_report
import cv2
from itertools import chain

%matplotlib inline

In [3]:
# Import custom functions from seperate workbook
from ipynb.fs.full.helper_functions import *

In [18]:
def create_train_som(x_train, n_features):
    
    # Create SOM dimensions
    som_nurons = int((math.sqrt(5*math.sqrt(n_features))))
#     print(som_nurons)
    x = som_nurons
    y = som_nurons
    
    #Create and train SOM
    som = MiniSom(x, y, n_features, sigma=0.3, learning_rate=0.5) # initialization of 6x6 SOM
    som.random_weights_init(x_train)
    print("Training...")
    som.train_random(x_train,100, verbose=False) # training with 100 iterations
    print("...ready!")
    return som

In [19]:
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data()

In [20]:
# Select a subset of images to train the SOM
image_list_train = x_train[0:1000]
image_list_test = x_test[0:1000]
image_list_res_test = y_test[0:1000]
image_list_res_train = y_train[0:1000]

In [21]:
# Here we have to flatten our data so we are able to feed it to the SOM
x_test_r = flatten_and_reshape(image_list_test)
x_train_r = flatten_and_reshape(image_list_train)

In [22]:
som = create_train_som(x_train_r, 784)

Training...
...ready!


In [23]:
distance_map = som.distance_map()

In [24]:
distance_map

array([[0.32879689, 0.51005443, 0.49372258, 0.56741816, 0.50097544,
        0.51456093, 0.53939002, 0.45505484, 0.45936083, 0.52610166,
        0.33945012],
       [0.53739866, 0.84269825, 0.89601312, 0.7671505 , 0.92222467,
        0.79617196, 0.73256427, 0.75308446, 0.77132337, 0.81845596,
        0.57490193],
       [0.55170266, 1.        , 0.71186579, 0.87742692, 0.85157369,
        0.70120936, 0.64521037, 0.82187442, 0.88642535, 0.89196143,
        0.56505198],
       [0.51319404, 0.74866177, 0.78180027, 0.95352028, 0.90362797,
        0.79866196, 0.78054678, 0.75524868, 0.91222536, 0.92016344,
        0.53483491],
       [0.47778643, 0.89376811, 0.77388448, 0.80200033, 0.91571686,
        0.87424071, 0.716983  , 0.80547858, 0.84452609, 0.75381258,
        0.5581976 ],
       [0.46219661, 0.66367452, 0.77283491, 0.88155411, 0.78131788,
        0.85202379, 0.83411983, 0.78781499, 0.80797132, 0.87296267,
        0.53106695],
       [0.46058439, 0.70049786, 0.87100637, 0.82433364, 0.

In [17]:
min(np.array(distance_map.flat)

SyntaxError: unexpected EOF while parsing (<ipython-input-17-11e1d07b02e2>, line 1)

The \_distance_from_weights function allows us to extract the distance map for the SOM and the given training example. The distance map contains the distance from each node within the SOM to the given training example. Using this we can extract the winning nodes distance to use in our convolutional layer.

So how do we creat the secondary layer? 

Ok we could do what the boss says, use the coordinants and the weight of the winning neuron for that example. 

what else / how would we index that. I mean we could just do exactly what we did last time index on observation number. 

What do we call this intermediate layer? whereby we combine two SOMs - convolutional layer? Well yeah beacuse were combining two seperate things. So maybe we need to work on thhat

OK step 1 - lets just create a very basic workflow using incredibly manual tasks. I mean lets just do it. 

In [25]:
x = som._distance_from_weights(x_train_r)

In [28]:
x

array([[1225.94669461, 1010.70187452,  711.45555026, ...,  997.6807901 ,
        1069.95438037,           nan],
       [          nan,           nan,           nan, ...,  527.01255892,
                  nan,           nan],
       [1973.27287486, 1663.7671456 , 1122.11719531, ..., 1520.12787533,
        2250.47663561, 1382.98588575],
       ...,
       [ 755.65522101,           nan,           nan, ..., 1266.66252103,
                  nan,           nan],
       [1055.25740128, 1163.65077494, 1187.42326068, ...,  941.188829  ,
        1184.25491051,  937.55397279],
       [1670.75613478,           nan,           nan, ...,  853.56621311,
        1751.24426507,           nan]])